In [1]:
import torch
import torch.nn as nn
import pandas as pd
from ofa.utils.layers import *
from ofa.utils.pytorch_modules import make_divisible

x:\Softwares\anaconda\envs\ml\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [5]:
df = pd.read_csv('datasets/cleaned_latency_summary.csv')
df.head(5)

,Unnamed: 0,Kernel Name,Number Of Runs,CU Full Name,Minimum Time (ms),Average Time (ms),Maximum Time (ms),Workload(GOP),DPU Performance(GOP/s),Mem IO(MB),Mem Bandwidth(MB/s)
0,1,ResidualBlock-in_96x14x14-out_136-k_7-e_3-s_1-...,500,DPUCZDX8G_1:batch-1,0.708,0.719,0.868,0.032,10.596,0.162,244.496
1,3,ResidualBlock-in_96x14x14-out_96-k_5-e_3-s_1-a...,500,DPUCZDX8G_1:batch-1,0.477,0.488,0.816,0.025,12.615,0.114,266.378
2,5,ResidualBlock-in_48x28x28-out_48-k_3-e_3-s_1-a...,500,DPUCZDX8G_1:batch-1,0.354,0.365,0.876,0.024,17.225,0.134,437.257
3,7,ResidualBlock-in_96x14x14-out_96-k_5-e_6-s_1-a...,500,DPUCZDX8G_1:batch-1,0.919,0.928,1.069,0.050,12.400,0.174,200.180
4,9,ResidualBlock-in_136x14x14-out_192-k_5-e_6-s_2...,500,DPUCZDX8G_1:batch-1,1.320,1.326,1.420,0.062,10.628,1.040,818.669


In [9]:

def build_name(cfg, k= None, e= None, multiplier= 1.2):
    
    layer_type = cfg[0]
    
    if layer_type == LinearLayer:
        in_channels = make_divisible(int(round(cfg[2] * multiplier)), 8)
        output_channels = cfg[3]
        name_parts = [
            'Linear',
            f"in-{in_channels}",
            f"out-{output_channels}",
        ]
        return '_'.join(name_parts)
        
    else:
        width, height = cfg[1], cfg[1]
        in_channels = cfg[2]
        output_channels = cfg[3]
        
        if in_channels > 8:
            in_channels = make_divisible(int(round(in_channels * multiplier)), 8)
            
        output_channels = make_divisible(int(round(output_channels * multiplier)), 8)
        
        input_shape_str = f"{in_channels}x{width}x{height}"
        
        if layer_type == ResidualBlock:
            stride = cfg[6]
            activation_function = cfg[7]
            use_se = cfg[8]
        
            name_parts = [
                "ResidualBlock",
                f"in_{input_shape_str}",
                f"out_{output_channels}",
                f"k_{k}",
                f"e_{e}",
                f"s_{stride}",
                f"act_{activation_function}",
                f"use_se_{use_se}"
            ]
            return '-'.join(name_parts)
            

            # the _, - I mistakely reversed thier order in conv and residual block in naming 
        elif layer_type == ConvLayer:
            name_parts = [
                'Conv',
                f"in-{input_shape_str}",
                f"out-{output_channels}",
                f"k-{cfg[4]}",
                f"act-{cfg[6]}",
            ]
            return '_'.join(name_parts)
    
    


In [10]:
base_resolution = 224

configurations = [
    (ConvLayer, base_resolution, 3, 16, 3, 2, "relu"),
    (
        ResidualBlock,
        base_resolution // 2,
        16,
        16,
        [1],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 2,
        16,
        24,
        [3, 4, 6],
        [3, 5, 7],
        2,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        24,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        24,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        24,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "relu",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 4,
        24,
        40,
        [3, 4, 6],
        [3, 5, 7],
        2,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        40,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        40,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        40,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "relu",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 8,
        40,
        80,
        [3, 4, 6],
        [3, 5, 7],
        2,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        80,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        80,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        80,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        False,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        80,
        112,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        112,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        112,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        112,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 16,
        112,
        160,
        [3, 4, 6],
        [3, 5, 7],
        2,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 32,
        160,
        160,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 32,
        160,
        160,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (
        ResidualBlock,
        base_resolution // 32,
        160,
        160,
        [3, 4, 6],
        [3, 5, 7],
        1,
        "h_swish",
        True,
    ),
    (ConvLayer, base_resolution // 32, 160, 960, 1, 1, "h_swish"),
    (ConvLayer, 1, 960, 1280, 1, 1, "h_swish"),
    (LinearLayer, 1, 1280, 1000, 1, 1),
]

     

Format of dictionary to store table 
 
latency_dict = 
``` 
{
'mobile_inverted_blocks': 
    
    [ index :
          {
            [(ks, e)]: 'latency'
        }  
        ],
    
    "other_blocks":{
        'layer_idx':'latency'       
    }
        
}
```

In [26]:
sucess = True

latency_dict = {
'mobile_inverted_blocks':[],
    
    "other_blocks":{     
    }    
}

for layer_idx, layer in enumerate(configurations): 
    if layer[0] is ResidualBlock:
        
        sub_dict = {}
        kernels = layer[5]
        expand = layer[4]
        for e in expand:
            for k in kernels:
                name = build_name(layer, k, e)
                mean_latency = df[ df['Kernel Name'] == name ].iloc[:,5].values
                
                sub_dict[(k,e)] = mean_latency[0]
                
                if len(mean_latency) == 0:
                    sucess = False
        latency_dict['mobile_inverted_blocks'].append(sub_dict)
                    
                    
    else:
        name = build_name(layer)
        mean_latency = df[ df['Kernel Name'] == name ].iloc[:,5].values
        if len(mean_latency) == 0:
            sucess = False
        latency_dict['other_blocks'][layer_idx] = mean_latency[0]

if sucess:
    print("Latency table build sucessful")
        

Latency table build sucessful


#### Saving the Dictionary 

In [29]:
import numpy as np 
np.save('datasets/latency_lut.npy', latency_dict)